In [1]:
import os
import sys
from time import sleep
from uuid import uuid4

import GCode
import GRBL
import numpy as np

# Laser Circles 

# Code:

In [10]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
if cnc.laser_mode is None:
    cnc.reset()
    sleep(0.5)
cnc.laser_mode = 1
sleep(0.5)
print("Laser Mode: {}".format(cnc.laser_mode))
if "Alarm" in cnc.status:
    cnc.home()

ok
Laser Mode: 1.0


In [11]:
def init():
    program = GCode.GCode()
    program.G0(F=500)
    program.G1(F=300)
    program.M3(S=1)
    program.G28()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G92(X=0, Y=0)
    program.M5()
    return program


def end():
    program = GCode.GCode()
    program.M5()
    program.G28()
    return program


In [14]:
def circle(center_x=10, center_y=10, radius=10, laser_pwm=255, clockwise=True):
    prog = GCode.GCode()
    prog.G0(X=(center_x - radius), Y=(center_y))
    prog.M3(S=laser_pwm)
    arc_cfg={
        "X": (center_x - radius),
        "Y": (center_y),
        "I": radius,
        "J": 0,      
    }
    if clockwise:
        prog.G2(**arc_cfg)
    else:
        prog.G3(**arc_cfg)
    prog.M5()
    return prog

def circle(center=(10, 10), radius=10, laser_pwm=255, clockwise=True, dynamic_power=False):
    (center_x, center_y) = center
    prog = GCode.GCode()
    prog.G0(X=(center_x - radius), Y=(center_y))
    if dynamic_power:
        prog.M4(S=laser_pwm)
    else:
        prog.M3(S=laser_pwm)
    arc_cfg={
        "X": (center_x - radius),
        "Y": (center_y),
        "I": radius,
        "J": 0,      
    }
    if clockwise:
        prog.G2(**arc_cfg)
    else:
        prog.G3(**arc_cfg)
    prog.M5()
    return prog


In [16]:
circle(center=(200,50), radius=10, laser_pwm=255,clockwise=True)

<GCode>[cmds=4]

In [20]:
prog=GCode.GCode(machine=cnc)
prog+=init()
prog.G0(Z=-16)
prog+=circle(center=(200,50), radius=10, laser_pwm=255,clockwise=True)
prog.M5()

In [21]:
prog.run()

In [ ]:
while True:
    for F in [50, 100, 200, 300]:
        prog=GCode.GCode(machine=cnc)
        prog.G0(Z=-16)
        prog.G1(F=F)
        prog+=circle(center=(200,50), radius=10, laser_pwm=255,clockwise=True)
        prog.run()

^C
